# 5G NR matrix ::: BG-1
# Lifting size Z = 8
# Using alist generated by alist_generate.m

In [1]:
import numpy as np

In [2]:
Z = 8 # Lifting size
N = 68 * Z # Variable nodes
m = 46 * Z # Check nodes

In [3]:
start_lines = 4 # Initial redundant information in alist
check2var_lines = start_lines + m # Scoop check2var lines in a list

# Read the alist

In [4]:
# with open(r'C:\Users\SIVA ADITYA GOOTY\Desktop\NR_1_0_8_alist.alist', 'r',) as file:
#     lines = file.read().splitlines()


# lines=lines[start_lines:check2var_lines] # skip the first 4 lines in the alist

# data=[]

# for i in range(m):

#     line = lines[i]
#     line_entry = lines[i].split()
#     line_len = len(line.split())
#     #val = np.zeros(line_len,dtype=int)
#     val=[]
#     for j in range(line_len):
#         val.append(int(line_entry[j]))
#     data.append(val)
# print(type(data[0][5]))

# Import the bipartite graph program

In [5]:
import fgSiva

In [6]:
class NR_1_0_8(fgSiva.Encoding):
    def __init__(self, seclength=1):
        
        with open(r'C:\Users\SIVA ADITYA GOOTY\Desktop\NR_1_0_8_alist.alist', 'r',) as file:
            lines = file.read().splitlines()


        lines=lines[start_lines:check2var_lines] # skip the first 4 lines in the alist

        self.Check2VarEdges=[]

        for i in range(m):

            line = lines[i]
            line_entry = lines[i].split()
            line_len = len(line.split())
            val = []

            for j in range(line_len):
                
                temp_val = int(line_entry[j])
                
                if temp_val != 0:
                    val.append(int(line_entry[j])) 
            
            self.Check2VarEdges.append(val)
        
        super().__init__(self.Check2VarEdges, None, seclength)
        #self.maxdepth = 12Size of parity check matrix

In [7]:
EbNo_dB = np.arange(-1,8,1)

# Run the encoding and decoding

In [ ]:
err = np.zeros(len(EbNo_dB))


for i in range(len(EbNo_dB)):
    n_iter = 0
    err_fixed = 0
    n_errs = 0
    while n_errs<=50:
        n_iter+=1

        # Initialize LDPC factor graph
        LDPCFactorGraph = NR_1_0_8()

        msg = np.zeros(N-m)

        # Encode codeword
        c = LDPCFactorGraph.encodemessage(msg)

        assert len(c) == N

        #print(c)



        nvar = N / ((10 ** (EbNo_dB[i] / 10.0)) * (N-m))
        #print("Variance",nvar)

        x = (2*c-1)                                       # modulate using binary phase shift keying (BPSK)
        n = np.random.normal(0,nvar,N)                    # generate random Gaussian noise
        y = x + n

        #print(y)


        # Note: it is good practice to reset the factor graph if you will be using it multiple times in a row
        # (e.g. averaging over multiple simulations)
        LDPCFactorGraph.reset()

        # Initialize variable nodes with observations
        # Note that varnodeid is starts with 1 and python utilizes 0-based indexing
        #LDPCFactorGraph.printgraphcontent()
        k=0
        for varnodeid in LDPCFactorGraph.varlist:
            k+=1
            Pxi_1 = (-2/nvar) * y[varnodeid-1]
            #print(Pxi_1)
            #print("Prob",Pxi_1)
            if k < 2: 
                LDPCFactorGraph.setobservation(varnodeid, 0)
            else:
                LDPCFactorGraph.setobservation(varnodeid, Pxi_1)
        # LDPCFactorGraph.printgraphcontent()
        # Run message-passing algorithm on graph
        numIterations = 10
        for idxiteration in range(numIterations):
            LDPCFactorGraph.updatechecks()
            LDPCFactorGraph.updatevars()
            #print("######",idxiteration)
            #LDPCFactorGraph.printgraphcontent()

        # Extract information from graph
        cHt = np.zeros(N)
        tmp = np.zeros(N)
        for varnodeid in LDPCFactorGraph.varlist:
            tmp[varnodeid-1] = LDPCFactorGraph.getextrinsicestimate(varnodeid)+LDPCFactorGraph.getobservation(varnodeid)
            if tmp[varnodeid-1]<0:
                tmp[varnodeid-1]=1
            else:
                tmp[varnodeid-1]=0
        
        if np.sum(c != tmp) != 0:
            n_errs+=1
            err_fixed+=(np.sum(c != tmp))    
    print(n_iter)
    err[i] = err_fixed/(n_iter*N)

51
51
51
51
51
58


# BER VS EbNo_dB plot

In [ ]:
import matplotlib.pyplot as plt
plt.semilogy(EbNo_dB,err,'r')
plt.xlabel('EbNo_dB')
plt.ylabel('Bit error rate : Codeword')
plt.title('20 Monte-Carlo simulations')
plt.ylim(0.00001,5)
plt.show()